In [23]:
#Los 3 primeros imports son para llamar a la API de ChatGPT
from dotenv import load_dotenv
import os #provides ways to access the Operating System and allows us to read the environment variables
import openai

import requests #Este import es para llamar a la API del tiempo

import pandas as pd #Este import es para cargar el CSV de latitud y longitud de todos los pueblos de españa

In [2]:
#Llamo a la API de chatGPT aunque ahora he decidido no empezar usandola

load_dotenv()
my_secret_key = os.getenv("my_openai_key")

In [3]:
#Llamo a la contraseña que tengo guardada en .env

openai.api_key = my_secret_key

In [4]:
#Le doy la bienvenida al usuario

print("Welcome to the APP that helps you to plan your road trip in Campervan starting from Madrid")

Welcome to the APP that helps you to plan your road trip in Campervan starting from Madrid


In [5]:
#Le pregunto al usuario cuantos días va a viajar ya que es la base de mi elección de itinerario

days = float(input("Please enter how many days are you going to travel: "))

Please enter how many days are you going to travel: 5


In [6]:
#Calculo cuantos KM va a viajar en base a los días (resto 1 porque el último día no lo tengo en cuenta)

number = (days - 1)*100
number

400.0

In [7]:
#Creo un diccionario en base al radio de KM para usarlo como referencia para saber el tiempo

dictionary = {
    'N': ['Riaza', 'Aranda de Duero', 'Burgos', 'Santander', 'Santander'],
    'NO': ['Segovia', 'Zamora', 'Ponferrada', 'Lugo', 'La Coruña'],
    'O': ['Avila', 'Salamanca', 'Ciudad Rodrigo', 'Ciudad Rodrigo', 'Ciudad Rodrigo'],
    'SO': ['Talavera de la Reina', 'Plasencia', 'Merida', 'Sevilla', 'Huelva'],
    'S': ['Toledo', 'Ciudad Real', 'Jaen', 'Granada', 'Malaga'],
    'SE': ['Tarancon', 'Albacete', 'Almansa', 'Murcia', 'Almeria'],
    'E': ['Sacedon', 'Cuenca', 'Valencia', 'Javea', 'Alicante'],
    'NE': ['Siguenza', 'Calatayud', 'Zaragoza', 'Huesca', 'Andorra']}

In [8]:
#Necesito restar los días -2 para que me de la posición correcta del diccionario ya que por ejemplo 
# en 3 días de viaje quiero hacer como máximo 200 kms, es decir la 2 posición que en python es 1

d = int(days - 2)

In [9]:
#Creo una función para que me genere automaticamente las ubicaciones en base a los días de viaje

def get_places(dictionary, d):
    values = {}
    for key, value_list in dictionary.items():
        if d < len(value_list):
            values[key] = value_list[d]
    return values

In [10]:
#Reviso que me devuelva el resultado correcto https://www.weatherapi.com/docs/

result = get_places(dictionary, d)
result

{'N': 'Santander',
 'NO': 'Lugo',
 'O': 'Ciudad Rodrigo',
 'SO': 'Sevilla',
 'S': 'Granada',
 'SE': 'Murcia',
 'E': 'Javea',
 'NE': 'Huesca'}

In [11]:
#Creo todas las variables para llamar a la API del tiempo menos "q" que representa el lugar 
#de dónde quiero ver la previsión del tiempo

url = 'http://api.weatherapi.com/v1'
forecast = '/forecast.json?'
key = 'key=1d638152f69742a6b19113935230107'
wdays = '&days=1'
final = '&aqi=no&alerts=no'

In [19]:
#Ahora creo una variable "q" para cada punto cardinal

q_var = '&q='
norte = q_var + result['N']
noroeste = q_var + result['NO']
oeste = q_var + result['O']
suroeste = q_var + result['SO']
sur = q_var + result['S']
sureste = q_var + result['SE']
este = q_var + result['E']
noreste = q_var + result['NE']

In [20]:
#Creo una llamada a la API para cada lugar

check_rain_north = requests.get(url + forecast + key + norte + wdays + final)
check_rain_nortwest = requests.get(url + forecast + key + noroeste + wdays + final)
check_rain_west = requests.get(url + forecast + key + oeste + wdays + final)
check_rain_southwest = requests.get(url + forecast + key + suroeste + wdays + final)
check_rain_south = requests.get(url + forecast + key + sur + wdays + final)
check_rain_southeast = requests.get(url + forecast + key + sureste + wdays + final)
check_rain_east = requests.get(url + forecast + key + este + wdays + final)
check_rain_northeast = requests.get(url + forecast + key + noreste + wdays + final)

In [14]:
check_rain_north_result = check_rain_north.json()
check_rain_nortwest_result = check_rain_nortwest.json()
check_rain_west_result = check_rain_west.json()
check_rain_southwest_result = check_rain_southwest.json() 
check_rain_south_result = check_rain_south.json() 
check_rain_southeast_result = check_rain_southeast.json()
check_rain_east_result = check_rain_east.json()
check_rain_northeast_result = check_rain_northeast.json()

In [15]:
print("Awesome! I suggest to travel to the following directions as the others could rain tomorrow")

#North
if check_rain_north_result['forecast']["forecastday"][0]['day']['daily_chance_of_rain'] == 0:
    print('North')
    
#Northwest
if check_rain_nortwest_result['forecast']["forecastday"][0]['day']['daily_chance_of_rain'] == 0:
    print('Northwest')

#West
if check_rain_west_result['forecast']["forecastday"][0]['day']['daily_chance_of_rain'] == 0:
    print('West')
    
#Southwest
if check_rain_southwest_result['forecast']["forecastday"][0]['day']['daily_chance_of_rain'] == 0:
    print('Southwest')
    
#South
if check_rain_south_result['forecast']["forecastday"][0]['day']['daily_chance_of_rain'] == 0:
    print('South')

#Southeast
if check_rain_southeast_result['forecast']["forecastday"][0]['day']['daily_chance_of_rain'] == 0:
    print('Southeast')

#East
if check_rain_east_result['forecast']["forecastday"][0]['day']['daily_chance_of_rain'] == 0:
    print('East')
    
#Northeast
if check_rain_northeast_result['forecast']["forecastday"][0]['day']['daily_chance_of_rain'] == 0:
    print('Northeast')

Awesome! I suggest to travel to the following directions as the others could rain tomorrow
Northwest
West
Southwest
South
Southeast


In [38]:
#Cargo el dataset de municipios para tener las coordenadas

municipios = pd.read_csv('./data/pueblos_bonitos.csv', encoding='latin-1', sep=';')
municipios[:2]

,Pueblo,Provincia,Longitud,Latitud,Datos de Interés
0,Alcalá del Júcar,Albacete,"-1,42932697","39,1932018",D.O. Pueblo Bonito
1,Almagro,Ciudad Real,"-3,71155855","38,88984134",D.O. Pueblo Bonito


In [21]:
#Ahora le pido al usuario que elija el punto cardinal

direction = input('Choose one direction: ')

Choose one direction: west


In [ ]:
if direction == 'west':
    

In [ ]:
input('Cool!/n In that case, these are the places I recommend')

In [ ]:
prompt = f"From Madrid, recommend the top 3 places to visit within {kilometers} kilometers based on popular opinions."

In [ ]:
variables = []

for i in result.keys():
    print('&q=' + result[i])

In [ ]:
q_ = result[input("Select N, NO...")] 

In [ ]:
q_

In [ ]:
example = response.json()
example

In [ ]:
example.keys()

In [ ]:
example['forecast'].keys()

In [ ]:
example['forecast']["forecastday"][0]['day']

In [ ]:
example['forecast']["forecastday"][0]['day']['daily_chance_of_rain']

In [31]:
kilometers = float(input("Enter the number of kilometers: "))

Enter the number of kilometers: 150


In [32]:
prompt = f"From Madrid, recommend the top 3 places to visit within {kilometers} kilometers based on popular opinions."

In [33]:
response = openai.Completion.create(
  engine="gpt-3.5-turbo",
  prompt=prompt,
  max_tokens=50,
  n=3,
  stop=None,
  temperature=0.5,
  top_p=1.0
)


RateLimitError: You exceeded your current quota, please check your plan and billing details.

In [29]:
import os
import openai
openai.api_key = os.getenv("my_openai_key")
openai.Model.list()


<OpenAIObject list at 0x2873f8019d0> JSON: {
  "object": "list",
  "data": [
    {
      "id": "whisper-1",
      "object": "model",
      "created": 1677532384,
      "owned_by": "openai-internal",
      "permission": [
        {
          "id": "modelperm-KlsZlfft3Gma8pI6A8rTnyjs",
          "object": "model_permission",
          "created": 1683912666,
          "allow_create_engine": false,
          "allow_sampling": true,
          "allow_logprobs": true,
          "allow_search_indices": false,
          "allow_view": true,
          "allow_fine_tuning": false,
          "organization": "*",
          "group": null,
          "is_blocking": false
        }
      ],
      "root": "whisper-1",
      "parent": null
    },
    {
      "id": "babbage",
      "object": "model",
      "created": 1649358449,
      "owned_by": "openai",
      "permission": [
        {
          "id": "modelperm-49FUp5v084tBB49tC4z8LPH5",
          "object": "model_permission",
          "created": 166908

In [ ]:
recommended_places = [choice['text'].strip() for choice in response['choices']]

In [ ]:
print(f"\nTop 3 places to visit from Madrid within {kilometers} kilometers:")

In [ ]:
for i, place in enumerate(recommended_places, start=1):
    print(f"{i}. {place}")